In [92]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega

# for configuring connection 
from configobj import ConfigObj
import os

from IPython.display import display_html
from sqlalchemy import create_engine

In [93]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5433
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'eicu'
    
#postgres_str =('postgresql://'+conn_info["sqluser"]+':root'+'@'+host+':'+port+'/'+database)

In [94]:
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres
Password: ········


In [95]:
patient = pd.read_sql('SELECT * FROM eicu.patient', con)
#diagnosis = pd.read_sql('SELECT * FROM diagnosis', con)
apache = pd.read_sql('SELECT * FROM eicu.apacheapsvar', con)

In [96]:
result1 = pd.merge(patient,
                 apache,
                 on="patientunitstayid", 
                 how='inner')

In [97]:
result=result1.drop([ 'ethnicity','wardid','apacheadmissiondx','admissionheight','hospitaladmittime24',
        'hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 
       'hospitaldischargelocation','unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 
       'admissionweight', 'dischargeweight', 'unitdischargetime24','meanbp',
       'unitdischargelocation', 'unitdischargestatus',
       'apacheapsvarid', 'intubated','dialysis','meds', 'hematocrit',
       'creatinine', 'albumin','pco2','glucose','hospitaldischargestatus','patientunitstayid',
        'patienthealthsystemstayid','uniquepid',
        'hospitaladmitoffset','hospitaldischargeoffset','unitdischargeoffset'],axis = 1)
print(result.columns)

Index(['gender', 'age', 'hospitalid', 'unitstaytype', 'vent', 'eyes', 'motor',
       'verbal', 'urine', 'wbc', 'temperature', 'respiratoryrate', 'sodium',
       'heartrate', 'ph', 'pao2', 'bun', 'bilirubin', 'fio2'],
      dtype='object')


In [127]:
result.shape

(171177, 19)

In [100]:
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix

X_temp = result


# Mean Imputation for Age
X_temp.replace({'age': {'': '0','> 89': '91'}},inplace=True)
X_temp = X_temp.astype({"age": np.int})
X_temp.replace({'age': {0: int(X_temp.age.mean())}},inplace=True)
# print(X_temp.age.unique())

# Removing age below 18
X_temp = X_temp[X_temp.age >= 18]

# Removing records with no gender given
X_temp = X_temp[X_temp.gender != '']


In [102]:
X_temp['unitstaytype'] = pd.DataFrame(X_temp.unitstaytype.map({'stepdown/other': 0, 'readmit for undo': 0, 'admit':0, 'pre-admit':0,'transfer':0, 'readmit':1}))

In [104]:
X_temp.dropna()

,gender,age,hospitalid,unitstaytype,vent,eyes,motor,verbal,urine,wbc,temperature,respiratoryrate,sodium,heartrate,ph,pao2,bun,bilirubin,fio2
0,Female,70,59,0,0,4,6,5,-1.0000,14.7,36.1,30.0,139.0,140.0,-1.000,-1.0,27.0,4.1,-1.0
1,Female,52,60,0,0,-1,-1,-1,-1.0000,-1.0,36.4,16.0,-1.0,88.0,-1.000,-1.0,-1.0,-1.0,-1.0
2,Male,68,73,0,0,3,6,4,-1.0000,14.1,39.3,36.0,134.0,118.0,-1.000,-1.0,31.0,0.4,-1.0
3,Male,71,67,0,0,4,6,5,-1.0000,-1.0,-1.0,25.0,-1.0,104.0,-1.000,-1.0,-1.0,-1.0,-1.0
4,Female,77,66,0,1,1,3,1,-1.0000,12.7,35.1,33.0,145.0,120.0,7.450,51.0,9.0,-1.0,100.0
5,Female,25,73,0,0,3,6,5,-1.0000,-1.0,36.7,37.0,-1.0,102.0,-1.000,-1.0,-1.0,-1.0,-1.0
6,Male,82,60,0,1,3,6,4,-1.0000,42.7,40.1,54.0,133.0,204.0,7.460,65.0,32.0,-1.0,21.0
7,Female,91,63,0,1,4,6,5,-1.0000,-1.0,36.4,32.0,-1.0,57.0,-1.000,-1.0,-1.0,-1.0,-1.0
8,Female,81,73,0,1,4,6,5,-1.0000,8.0,34.8,4.0,-1.0,114.0,7.390,142.0,-1.0,-1.0,60.0
9,Male,59,73,0,0,4,6,5,-1.0000,4.1,37.2,10.0,142.0,114.0,-1.000,-1.0,8.0,-1.0,-1.0


In [105]:
ust = pd.DataFrame(X_temp['unitstaytype'])
X_temp.drop(['unitstaytype'],axis=1,inplace=True)

X_cat = X_temp.select_dtypes(exclude=['int64', 'float64'])                                                                                                         
X_dog = X_temp.select_dtypes(include=['int64', 'float64'])                                                                                                         


X_cat = pd.get_dummies(X_cat)                                                                                                                                 

X_temp=pd.concat([X_cat,X_dog,ust],axis=1)

#ust = pd.DataFrame(X_temp.unitstaytype.map({'stepdown/other': 0, 'readmit for undo': 0, 'admit':0, 'pre-admit':0,'transfer':0, 'readmit':1}))
X_temp['unitstaytype'] = ust['unitstaytype']
#X_temp.dropna()

In [115]:
X_temp.head()

,gender_Female,gender_Male,gender_Other,gender_Unknown,age,hospitalid,vent,eyes,motor,verbal,...,temperature,respiratoryrate,sodium,heartrate,ph,pao2,bun,bilirubin,fio2,unitstaytype
0,1,0,0,0,70,59,0,4,6,5,...,36.1,30.0,139.0,140.0,-1.00,-1.0,27.0,4.1,-1.0,0
1,1,0,0,0,52,60,0,-1,-1,-1,...,36.4,16.0,-1.0,88.0,-1.00,-1.0,-1.0,-1.0,-1.0,0
2,0,1,0,0,68,73,0,3,6,4,...,39.3,36.0,134.0,118.0,-1.00,-1.0,31.0,0.4,-1.0,0
3,0,1,0,0,71,67,0,4,6,5,...,-1.0,25.0,-1.0,104.0,-1.00,-1.0,-1.0,-1.0,-1.0,0
4,1,0,0,0,77,66,1,1,3,1,...,35.1,33.0,145.0,120.0,7.45,51.0,9.0,-1.0,100.0,0


In [112]:
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
from sklearn.metrics import roc_auc_score

In [113]:
X_readmission = X_temp.loc[X_temp['unitstaytype'] == 1]
X_other = X_temp.loc[X_temp['unitstaytype'] == 0]

print(X_readmission.shape,X_other.shape)

# X_alive_restricted = X_alive.tail(16000)

# X_temp = X_alive_restricted.append(X_expired)
X = pd.DataFrame(X_temp.drop(['unitstaytype'],axis=1))
#X = pd.DataFrame(X_temp.drop(['patientunitstayid'],axis=1))
Y = pd.DataFrame(X_temp['unitstaytype'])                
#X = pd.DataFrame(X.drop(['unitstaytype'],axis=1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, stratify=Y)

log_regressor   = LogisticRegression(penalty='l1', max_iter = 1e8, class_weight='balanced')
# log_regressor = GridSearchCV(estimator=log_regressor, param_grid={'C':np.logspace(-4,4,10)}, cv= 5)
log_regressor.fit(X_train, y_train)

pred =pd.DataFrame(log_regressor.predict(X_test),columns=["a"])
log_f1_score=(f1_score(y_test,pred))

print(confusion_matrix(y_test,pred))
print(result2.columns)
print(log_regressor.coef_)

print(log_f1_score)
auroc = roc_auc_score(y_test, pred)
print(auroc)

(9929, 22) (160706, 22)
[[20266 11875]
 [  954  1032]]
Index(['gender', 'age', 'hospitalid', 'unitstaytype', 'vent', 'eyes', 'motor',
       'verbal', 'urine', 'wbc', 'temperature', 'respiratoryrate', 'sodium',
       'heartrate', 'ph', 'pao2', 'bun', 'bilirubin', 'fio2',
       'numbedscategory', 'teachingstatus', 'region'],
      dtype='object')
[[ 2.23717214e-01  2.94579428e-01  0.00000000e+00 -2.78862689e+00
   5.50688026e-03 -1.03063090e-03  1.94318845e-01  4.68906588e-02
  -1.35777218e-02 -1.31977936e-01 -1.14027259e-05  1.00863310e-02
  -1.11170476e-02  2.06012107e-03 -4.06144561e-03  2.42879120e-03
  -1.82973606e-02 -1.46187401e-03  6.27544153e-03  1.69062862e-02
   6.64567093e-03]]
0.13858859867051637
0.5750858354394078


In [116]:
from xgboost import XGBClassifier
from keras.utils import to_categorical
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

In [117]:
hospital = pd.read_sql('Select * from eicu.hospital',con)
print(patient.columns)

result2 = pd.merge(result,
                 hospital,
                 on="hospitalid", 
                 how='inner')

print(result2.columns)

Index(['patientunitstayid', 'patienthealthsystemstayid', 'gender', 'age',
       'ethnicity', 'hospitalid', 'wardid', 'apacheadmissiondx',
       'admissionheight', 'hospitaladmittime24', 'hospitaladmitoffset',
       'hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 'hospitaldischargeoffset',
       'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
       'admissionweight', 'dischargeweight', 'unitdischargetime24',
       'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',
       'uniquepid'],
      dtype='object')
Index(['gender', 'age', 'hospitalid', 'unitstaytype', 'vent', 'eyes', 'motor',
       'verbal', 'urine', 'wbc', 'temperature', 'respiratoryrate', 'sodium',
       'heartrate', 'ph', 'pao2', 'bun', 'bilirubin', 'fio2',
       'numbedscategory', 'teachingstatus', 'region'],
      dtype='object')


In [123]:
X_temp1 = result2
X_temp1 = X_temp1[X_temp1.unitstaytype != '']
# X_temp=X_temp.dropna()

# Mean Imputation for Age
X_temp1.replace({'age': {'': '0','> 89': '91'}},inplace=True)
X_temp1 = X_temp1.astype({"age": np.int})
X_temp1.replace({'age': {0: int(X_temp1.age.mean())}},inplace=True)
# print(X_temp.age.unique())

# Removing age below 18
X_temp1 = X_temp1[X_temp1.age >= 18]

# Removing records with no gender given
X_temp1 = X_temp1[X_temp1.gender != '']

X_temp1['unitstaytype'] = pd.DataFrame(X_temp1.unitstaytype.map({'stepdown/other': 0, 'readmit for undo': 0, 'admit':0, 'pre-admit':0,'transfer':0, 'readmit':1}))
#X_temp1['unitstaytype'] = pd.DataFrame(X_temp.unitstaytype.map({'stepdown/other': 0, 'readmit for undo': 0, 'admit':0, 'pre-admit':0,'transfer':0, 'readmit':1}))
#X_temp['unitstaytype'] = ust['unitstaytype']



In [124]:
X_temp1.head()

,gender,age,hospitalid,unitstaytype,vent,eyes,motor,verbal,urine,wbc,...,sodium,heartrate,ph,pao2,bun,bilirubin,fio2,numbedscategory,teachingstatus,region
0,Female,70,59,0,0,4,6,5,-1.0,14.7,...,139.0,140.0,-1.0,-1.0,27.0,4.1,-1.0,<100,False,Midwest
1,Male,19,59,0,0,-1,-1,-1,-1.0,-1.0,...,-1.0,60.0,-1.0,-1.0,-1.0,-1.0,-1.0,<100,False,Midwest
2,Male,70,59,0,1,4,6,5,-1.0,12.8,...,140.0,120.0,-1.0,-1.0,48.0,-1.0,-1.0,<100,False,Midwest
3,Male,75,59,0,0,4,6,5,-1.0,9.8,...,139.0,94.0,-1.0,-1.0,13.0,-1.0,-1.0,<100,False,Midwest
4,Female,79,59,0,1,4,6,5,-1.0,13.5,...,140.0,94.0,-1.0,-1.0,15.0,0.8,-1.0,<100,False,Midwest


In [125]:
X_readmission = X_temp1.loc[X_temp1['unitstaytype'] == 1]
X_other = X_temp1.loc[X_temp1['unitstaytype'] == 0]

# X_alive_restricted = X_alive.tail(16000)

# X_temp1 = X_alive_restricted.append(X_expired)

#X = pd.DataFrame(X_temp1.drop(['patientunitstayid'],axis=1))
Y = pd.DataFrame(X_temp1['unitstaytype'])                
X = pd.DataFrame(X_temp.drop(['unitstaytype'],axis=1))

#X = pd.DataFrame(X.drop(['numbedscategory'],axis=1))

X_cat = X.select_dtypes(exclude=['int64', 'float64'])                                                                                                         
X_dog = X.select_dtypes(include=['int64', 'float64'])                                                                                                         

X_cat = pd.get_dummies(X_cat)                                                                                                                                 

X=pd.concat([X_cat,X_dog],axis=1)

print(X.head())

# Y = pd.DataFrame(Y.hospitaldischargestatus.map({'Alive': 1, 'Expired': 0}))

# print(X.head())
# print()
negative_class = len(X_readmission.index)
positivue_class = len(X_other.index)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y.iloc[:,0].values)

print(X_train.columns)
s = [100,500,600,900,1250]
loss_dict = {}
#weight = positivue_class/negative_class
for i in s:
    xgb_c = XGBClassifier(max_depth=3, n_estimators=i, n_jobs=-1)#,scale_pos_weight=weight)
    xgb_c.fit(X_train, Y_train,)
    pred = xgb_c.predict(X_test)
    
    print(confusion_matrix(Y_test,pred))
    
    f1 = f1_score(Y_test,pred)
    print(f1)
    y = to_categorical(Y_test,2)
    pred = to_categorical(pred,2)
    y=y.astype(np.uint8)
    
    Loss = log_loss(y,pred)
    print(Loss)
    loss_dict[i]= Loss

   gender_Female  gender_Male  gender_Other  gender_Unknown  age  hospitalid  \
0              1            0             0               0   70          59   
1              1            0             0               0   52          60   
2              0            1             0               0   68          73   
3              0            1             0               0   71          67   
4              1            0             0               0   77          66   

   vent  eyes  motor  verbal  ...     wbc  temperature  respiratoryrate  \
0     0     4      6       5  ...    14.7         36.1             30.0   
1     0    -1     -1      -1  ...    -1.0         36.4             16.0   
2     0     3      6       4  ...    14.1         39.3             36.0   
3     0     4      6       5  ...    -1.0         -1.0             25.0   
4     1     1      3       1  ...    12.7         35.1             33.0   

   sodium  heartrate    ph  pao2   bun  bilirubin   fio2  
0   139.0